# making an aggregate master dataframe for baseline model
this notebook is going to make standardized longformat dataframes for each dataframe that i will adjust for each model to. 

for my first pass, i will work to establish a baseline model by using:

the single "worst", or value that most indicates poor clinical outcomes, for each variable so each variable only has one row per patient. 

from my notes:

- Step1: establish a baseline (ie train model initially), using an obvious baseline: last valid mesurement of a particular variable.   Will hope that it doesn’t perform too good or too bad. Second would be to pick an aggregate within a time window (over 3 days, or of each day, ie can change graunlarity).

- Next try temporal trend, maybe vector autoregression. 

So my first step would be to pick either the last recorded value for each variable or the ‘worst’ value, or ones that we might expect to indicate poor outcome (NEED TO CHOOSE)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 119 ms


In [100]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.3 s


In [ ]:
#import all clinical variables
##ensure they are the versions with UOM

In [12]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '22102018'

ventcategory_df=pd.read_csv(Path(
    wd+'/data/processed/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/processed/%s_vaso_dose_72_rmout.csv'  %(date)), index_col=0)


echodata_72=pd.read_csv(Path(
    wd+'/data/processed/%s_echodata_72_df.csv' %(date)), index_col=0)

labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/processed/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/processed/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/processed/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/processed/%s_vitals_all_nosummary_nonull.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/processed/%s_uti_all_72_notnull.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/processed/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/processed/%s_rrt_merged_allpt_df_notnull.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/processed/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/processed/%s_sofa_score_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 19.7 s


10/5/18
# Standardizing Columns in df to merge together

10/22/18:
unit importing notebook has adjusted units, need to go through each individually and see which still needs work.

### list of columns in master df:
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- dtype

In [52]:
#making a day column
def day_col(df):
    day1= (
        (pd.to_datetime(df['charttime'])-pd.to_datetime(df['t_0']))
           .between('00:00:00','23:59:59'))
    day2= (
        (pd.to_datetime(df['charttime'])-pd.to_datetime(df['t_0']))
           .between('24:00:00','47:59:59'))
    day3= (
        (pd.to_datetime(df['charttime'])-pd.to_datetime(df['t_0']))
           .between('48:00:00','71:59:59'))
    
    
    df['day']= None
    
    df.loc[day1, 'day']= 1
    df.loc[day2, 'day']= 2
    df.loc[day3, 'day']= 3
    
    return(df)

time: 12.1 ms


In [53]:
day_col(vaso_dose_72)

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta,std,mean,high_value,day
0,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000,1.043056,0.161787,0.5,1
1,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000,1.043056,0.161787,0.5,1
2,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000,1.043056,0.161787,0.5,3
3,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000,1.043056,0.161787,0.5,3
4,201005.0,2169-10-25 17:25:00,2169-10-25 18:00:00,0.020000,NaN,NaN,mcgkgmin,epinephrine,2169-10-25,0 days 17:25:00.000000000,0.110696,0.055967,0.5,1
5,201005.0,2169-10-25 18:30:00,2169-10-25 19:00:00,0.020000,NaN,NaN,mcgkgmin,epinephrine,2169-10-25,0 days 18:30:00.000000000,0.110696,0.055967,0.5,1
6,201358.0,2151-10-21 13:45:00,2151-10-22 00:50:00,0.040021,1.577751,mg,mcg/kg/min,epinephrine,2151-10-20,1 days 13:45:00.000000000,1.043056,0.161787,0.5,2
7,201906.0,2178-12-27 02:04:00,2178-12-27 11:50:00,0.140292,1.647079,mg,mcg/kg/min,epinephrine,2178-12-26,1 days 02:04:00.000000000,1.043056,0.161787,0.5,2
8,202253.0,2131-10-17 03:10:00,2131-10-17 03:15:00,0.100000,NaN,NaN,mcgkgmin,epinephrine,2131-10-17,0 days 03:10:00.000000000,0.110696,0.055967,0.5,1
9,202253.0,2131-10-17 03:30:00,2131-10-17 04:00:00,0.100000,NaN,NaN,mcgkgmin,epinephrine,2131-10-17,0 days 03:30:00.000000000,0.110696,0.055967,0.5,1


time: 146 ms


In [58]:
vaso_dose_72.loc[vaso_dose_72.loc[:,'day']=='None',:] #all good

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta,std,mean,high_value,day


time: 11.4 ms


In [57]:
vaso_dose_72['day'].value_counts()

1    26032
2    16831
3     9685
Name: day, dtype: int64

time: 10.9 ms


In [ ]:
# stopped here, need to add day column to the rest of the sheets. 

### Vaso_dose

In [14]:
vaso_dose_72.head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta,std,mean,high_value
0,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000,1.043056,0.161787,0.5
1,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000,1.043056,0.161787,0.5
2,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000,1.043056,0.161787,0.5
3,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000,1.043056,0.161787,0.5
4,201005.0,2169-10-25 17:25:00,2169-10-25 18:00:00,0.020000,NaN,NaN,mcgkgmin,epinephrine,2169-10-25,0 days 17:25:00.000000000,0.110696,0.055967,0.5


time: 20.2 ms


# labs_all_nosummary_72
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [179]:
#labs_all_nosummary_72
labs_all_nosummary_72.head(20)

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom,endtime
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L,NaN
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L,NaN
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L,NaN
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L,NaN
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L,NaN
5,9,150750,220597,2149-11-14 06:00:00,SODIUM,137.0,2149-11-13,1 days 06:00:00.000000000,mEq/L,NaN
6,19,109235,273430,2108-08-06 01:08:00,SODIUM,138.0,2108-08-06,0 days 01:08:00.000000000,mEq/L,NaN
7,19,109235,273430,2108-08-07 05:07:00,SODIUM,139.0,2108-08-06,1 days 05:07:00.000000000,mEq/L,NaN
8,19,109235,273430,2108-08-08 05:20:00,SODIUM,141.0,2108-08-06,2 days 05:20:00.000000000,mEq/L,NaN
9,21,111970,216859,2135-02-05 04:48:00,SODIUM,134.0,2135-02-05,0 days 04:48:00.000000000,mEq/L,NaN


time: 17.8 ms


In [137]:
labs_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom,endtime
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L,NaN
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L,NaN
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L,NaN
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L,NaN
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L,NaN


time: 13.1 ms


# ventcategory_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [140]:
ventcategory_df_melt = pd.melt(ventcategory_df, id_vars=['icustay_id','t_0'], var_name='label')
ventcategory_df_melt.head()

,icustay_id,t_0,label,value
0,267059.0,2182-11-12,vent_day1,Mech
1,278729.0,2159-11-08,vent_day1,Oxygen
2,219670.0,2190-08-01,vent_day1,Oxygen
3,295185.0,2154-11-11,vent_day1,Oxygen
4,211472.0,2193-12-11,vent_day1,None


time: 23.2 ms


In [162]:
# #adding in charttime

# ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day1','charttime']=(
#     pd.to_datetime(
#         ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day1','t_0'])
#             +pd.Timedelta(days=1))

# ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day2','charttime']=(
#     pd.to_datetime(
#         ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day2','t_0'])
#             +pd.Timedelta(days=2))

# ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day3','charttime']=(
#     pd.to_datetime(
#         ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day3','t_0'])
#             +pd.Timedelta(days=3))

# ventcategory_df_melt['charttime']= pd.to_datetime(ventcategory_df_melt['charttime'])

time: 113 ms


# weightfirstday_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [166]:
weightfirstday_df_melt= pd.melt(weightfirstday_df, id_vars='icustay_id', var_name='label')
weightfirstday_df_melt.head()

,icustay_id,label,value
0,200012,weight,51.2
1,200014,weight,62.0
2,200019,weight,65.0
3,200030,weight,113.6
4,200033,weight,74.0


time: 15 ms


# heightfirstday_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
heightfirstday_df_melt= pd.melt(heightfirstday_df, id_vars='icustay_id', var_name='label')
heightfirstday_df_melt.head()

# vitals_all_nosummary_nonull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values
vitals_all_nosummary_nonull.head()

# uti_all_72_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [13]:
uti_all_72.head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
60,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99831,2131-05-21,0 days 01:33:00.000000000
61,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99811,2131-05-21,0 days 01:33:00.000000000
62,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41511,2131-05-21,0 days 01:33:00.000000000
63,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4538,2131-05-21,0 days 01:33:00.000000000
64,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99672,2131-05-21,0 days 01:33:00.000000000


time: 103 ms


# bg_all_nosummary_72
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#na values- removing all null values from value.
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72.head()

# rrt_merged_allpt_df_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#na values
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull

# gcs72_df_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#need to reshape the dataframe
gcs72_df_melt = pd.melt(gcs72_df, id_vars=['subject_id','hadm_id','icustay_id','t_0','ICU_admit','admit_plus_day','delta'], var_name='label')

gcs72_df_melt_notnull= gcs72_df_melt[gcs72_df_melt['value'].notnull()] #62772 ->62650
gcs72_df_melt_notnull.head()

# sofa_df_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#need to reshape the dataframe
sofa_df_melt = pd.melt(sofa_df_72, id_vars=['subject_id','hadm_id','icustay_id','t_0','day','ICU_admit','approx_charttime','floor_charttime'], var_name='label')

sofa_df_melt_notnull= sofa_df_melt[sofa_df_melt['value'].notnull()] #606651 ->302459
sofa_df_melt_notnull.head()